In [4]:
import pandas as pd
from xgboost.sklearn import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
%matplotlib inline

ModuleNotFoundError: No module named 'xgboost'

In [6]:
df_train = pd.read_csv(r'C:\Users\CAMNG3\Downloads\Data\raw_data_train.csv')
df_test = pd.read_csv(r'C:\Users\CAMNG3\Downloads\Data\raw_data_test.csv')


In [7]:
##**Converting the dtype of ' Stay_In_Current_City_Years'**
###(convert obj intto int)
df_train['Stay_In_Current_City_Years'] = df_train['Stay_In_Current_City_Years'].str.replace('+','')
df_train['Stay_In_Current_City_Years'] = df_train['Stay_In_Current_City_Years'].astype(int)

df_test['Stay_In_Current_City_Years'] = df_test['Stay_In_Current_City_Years'].str.replace('+','')
df_test['Stay_In_Current_City_Years'] = df_test['Stay_In_Current_City_Years'].astype(int)

In [68]:
## Dropping User_id and Product_ID
df_train=df_train.drop('User_ID',axis=1)
df_test=df_test.drop('User_ID',axis=1)
df_train=df_train.drop('Product_ID',axis=1)
df_test=df_test.drop('Product_ID',axis=1)

In [69]:
df_train=df_train.drop('Product_Category_3',axis=1)
df_test=df_test.drop('Product_Category_3',axis=1)

In [70]:
## Imputing missing values with mode
print('Unique values: ',df_train['Product_Category_2'].unique())
df_train['Product_Category_2'].mode()[0]
df_train['Product_Category_2']=df_train['Product_Category_2'].fillna(df_train['Product_Category_2'].mode()[0])
df_train.isnull().sum()

Unique values:  [nan  6. 14.  2.  8. 15. 16. 11.  5.  3.  4. 12.  9. 10. 17. 13.  7. 18.]


Gender                        0
Age                           0
Occupation                    0
City_Category                 0
Stay_In_Current_City_Years    0
Marital_Status                0
Product_Category_1            0
Product_Category_2            0
Purchase                      0
dtype: int64

In [71]:
print('Unique values: ',df_test['Product_Category_2'].unique())
df_test['Product_Category_2'].mode()[0]
df_test['Product_Category_2']=df_test['Product_Category_2'].fillna(df_test['Product_Category_2'].mode()[0])
df_test.isnull().sum()

Unique values:  [11.  5. 14.  9.  3.  4. 13.  2. nan  8. 15. 17. 12. 16.  6. 10. 18.  7.]


Gender                        0
Age                           0
Occupation                    0
City_Category                 0
Stay_In_Current_City_Years    0
Marital_Status                0
Product_Category_1            0
Product_Category_2            0
dtype: int64

In [72]:
df_train['Gender']=df_train['Gender'].map({'F':0,'M':1}) #categorical to integer
df_test['Gender']=df_test['Gender'].map({'F':0,'M':1}) #categorical to integer
    

In [73]:
cols = ['Age', 'City_Category', 'Stay_In_Current_City_Years']
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in cols:
    df_train[col] = le.fit_transform(df_train[col])
df_train.head()

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Purchase
0,0,0,10,0,2,0,3,8.0,8370
1,0,0,10,0,2,0,1,6.0,15200
2,0,0,10,0,2,0,12,8.0,1422
3,0,0,10,0,2,0,12,14.0,1057
4,1,6,16,2,4,0,8,8.0,7969


In [74]:
cols = ['Age', 'City_Category', 'Stay_In_Current_City_Years']
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in cols:
    df_test[col] = le.fit_transform(df_test[col])
df_test.head()

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2
0,1,4,7,1,2,1,1,11.0
1,1,2,17,2,0,0,3,5.0
2,0,3,1,1,4,1,5,14.0
3,0,3,1,1,4,1,4,9.0
4,0,2,1,2,1,0,4,5.0


In [75]:
## Splitting Train and Test data
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df_train,df_train['Purchase'],test_size=0.25,random_state=0)

In [76]:
x_train.shape,y_train.shape


((412551, 9), (412551,))

In [77]:
import numpy as np
features=['Age','Gender','City_Category','Product_Category_1','Product_Category_2']

for feature in features:
    labels_ordered=df_train.groupby([feature])['Purchase'].mean().sort_values().index
    labels_ordered={k:i for i,k in enumerate(labels_ordered,0)}
    df_train[feature]=df_train[feature].map(labels_ordered)

In [78]:
df_train=df_train[df_train.Product_Category_1 > 2.5]
df_train.reset_index(drop=True,inplace=True)

In [79]:
feature_scale=[feature for feature in df_train.columns if feature not in ["Purchase","Stay_In_Current_City_Years","Age"]]

from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(df_train[feature_scale])

MinMaxScaler()

In [80]:
import numpy as np
features=['Age','Gender','City_Category','Product_Category_1','Product_Category_2']

for feature in features:
    labels_ordered=df_train.groupby([feature])['Purchase'].mean().sort_values().index
    labels_ordered={k:i for i,k in enumerate(labels_ordered,0)}
    df_train[feature]=df_train[feature].map(labels_ordered)

In [81]:
##for feature selection
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)

In [82]:
df_train=df_train[df_train.Product_Category_1 > 2.5]
df_train.reset_index(drop=True,inplace=True)
df_train["Purchase"]=np.log1p(df_train["Purchase"])

X=df_train.drop('Purchase',axis=1)
y=df_train['Purchase']

In [83]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [84]:
from xgboost.sklearn import XGBRegressor
xgb_reg = XGBRegressor(learning_rate=1.0, max_depth=6, min_child_weight=40, seed=0)

xgb_reg.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=1.0, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=40, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [85]:
xgb_y_pred = xgb_reg.predict(X_test)
print('MAE',mean_absolute_error(y_test, xgb_y_pred))
print('MSE',mean_squared_error(y_test, xgb_y_pred))
print('R2_Score',r2_score(y_test, xgb_y_pred))
from math import sqrt
print("RMSE of XGBoost Model is ",sqrt(mean_squared_error(y_test, xgb_y_pred)))

MAE 0.2737730009796704
MSE 0.13522633546543616
R2_Score 0.5365340040156179
RMSE of XGBoost Model is  0.36773133598516755
